<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Rozbudowa-i-optymalizacja-similarity/colabs/Optymalizacja_i_rozbudowa_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wnioski

Przetestowane zostały dwa podejścia liczenia similarity:
1.   podstawowe korzystające z pythonowego liczenia similarity na listach;
2.   alternatywne korzystające z biblioteki Pandas.

W trakcie testów podstawowe podejście okazało się znacznie szybsze ($\sim 3$ razy szybsze). Czas wykonywania pierwszej metody to $3-3.5$ minuty, a drugiej metody to ponad $10$ minut.

Wybrana (najszybsza) metoda `similarity_one_vs_all` została z kolei dodatkowo usprawniona i z czasu wykonywania wynoszącego od $3$ do $3.5$ minuty zdołaliśmy osiągnąć czas wykonywania w granicach od $1.3$ do $1.7$ minuty, co jest dużym (dwukrotnym) przyśpieszeniem szybkości działania metody.



# Uwagi

W trakcie ładowania danych możemy zaznaczyć które kolumny są importowane (patrz kod w podrozdziale "Załadowanie danych z pliku o rozszerzeniu .parquet"), co potencjalnie zaoszczędza wykorzystywaną pamięć RAM w środowisku wykonwczym.

Metody liczące similairty korzystające z pythonowego podejścia do ich liczenia dostały opcję wpisania parametru `reduced_dataset` który domyślnie ustawiony jest na wartość `False`. Jeżeli decydujemy się na załadowanie jedynie niezbędnych do liczenia similarity kolumn, to parametr ten trzeba ustawić na wartość `True`. W przeciwnym przypadku funkcja licząca similarity będzie zwracała błąd i nie będzie się wykonywać.

# Przygotowanie notatnika

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1769, done.
remote: Counting objects: 100% (1769/1769), done.
remote: Compressing objects: 100% (751/751), done.
remote: Total 1769 (delta 1096), reused 1651 (delta 999), pack-reused 0
Receiving objects: 100% (1769/1769), 6.06 MiB | 27.95 MiB/s, done.
Resolving deltas: 100% (1096/1096), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout Rozbudowa-i-optymalizacja-similarity

Branch 'Rozbudowa-i-optymalizacja-similarity' set up to track remote branch 'Rozbudowa-i-optymalizacja-similarity' from 'origin'.
Switched to a new branch 'Rozbudowa-i-optymalizacja-similarity'


## Załadowanie danych z pliku o rozszerzeniu .parquet

In [4]:
from lib.const import JOINED_DATA

W poradniku "Tworzenie skrotu do GDrive" w folderze tutorials jest poradnik jak sprawić żeby poniższa lokalizacja się poprawnie ładowała.

Za pomocą parametru `columns` możemy ustalić które kolumny mają zostać pobrane z naszego zbioru.


UWAGA! Aby metoy dobrze działały na pomniejszonym zbiorze danych, to lista kolumn musi wyglądać tak jak w kodzie poniżej.

In [5]:
data = pd.read_parquet(JOINED_DATA, columns = ["nconst", "tconst", "titleType", "genres", "category", "primaryName"])
# TODO przepiąć się na nowszy plik gdy będzie on już zapisany na dysku

In [6]:
data.head(1)

,nconst,tconst,titleType,genres,category,primaryName
0,nm0000004,"[tt0694438, tt0694654, tt0694932, tt0694936, t...","[tvEpisode, tvEpisode, tvEpisode, tvEpisode, t...","[Comedy, Music, Comedy, Music, Comedy, Music, ...",actor,John Belushi


In [7]:
sample = data.sample(n = 10_000, random_state=1)

# Podstawowa metoda liczenia similarity

In [8]:
from typing import List, Any, Tuple
import pandas as pd
import numpy

In [9]:
from lib.similarity_utils import *

Wczytujemy dane o aktorze i zamieniamy je na odpowiedni format.

In [10]:
Arnold_schwarzenegger = prepare_pandas_row(find_actor(data, 'nm0000216'))

Za pomocą metody `similarity_one_vs_all` liczymy podobieństwa pomiędzy jednym aktorem, a resztą aktorów z wybranego przez nas zbioru danych. Najpierw znajdziemy $3$ aktorów ze zbioru, którzy są najbardziej podobni do Arnolda Schwarzeneggera.

Przy liczeniu similarity dla obciętego do niezbędnych kolumn zbioru danych, ustalamy parametr `reduced_dataset` w metodzie `similarity_one_vs_all` na `True`.

Testujemy czy metoda działa na zbiorze próbkowym.

In [11]:
%%time
ids, values = similarity_one_vs_all(sample, Arnold_schwarzenegger, reduced_dataset=True)

CPU times: user 435 ms, sys: 39.7 ms, total: 475 ms
Wall time: 454 ms


Szukamy $3$ najpodobniejszych aktorów z całego zbioru.

In [12]:
ranking = get_ranking(data, 'nm0000216', 3, reduced_dataset=True)
print(ranking)

('nm0001598', 'nm0001592', 'nm0000665')


Zamieniamy listę z `nconst` na imiona aktorów.

In [13]:
ranking_imion = replace_ids_with_names(data, ranking, reduced_dataset=True)
print(ranking_imion)

['Robert Patrick', 'Joe Pantoliano', 'D.B. Sweeney']


# Alternatywna metoda liczenia similarity

Znajdujemy aktora i zostawiamy jego dane w postaci `pd.DataFrame`.

In [ ]:
Arnold_schwarzenegger_pd = find_actor(data, 'nm0000216')

In [ ]:
Arnold_schwarzenegger_pd

nconst                                               nm0000216
tconst       [tt0233469, tt0364056, tt0711516, tt15739442, ...
titleType    [movie, videoGame, tvEpisode, video, tvEpisode...
genres       [Action, Drama, Thriller, Action, Adventure, S...
category                                                 actor
Name: 1651659, dtype: object

## Test metody na próbce

In [ ]:
%%time
ids_pd, values_pd = similarity_one_vs_all_pandas(sample, Arnold_schwarzenegger_pd)

CPU times: user 2.89 s, sys: 65.1 ms, total: 2.95 s
Wall time: 2.93 s


In [ ]:
values_pd[:3]

[-0.92, -0.45999999999999996, -0.47]

# Testy i porównanie metod na podstawowych danych

In [ ]:
%%time
ids1, val1 = similarity_one_vs_all(data, Arnold_schwarzenegger, reduced_dataset=True)

CPU times: user 1min 27s, sys: 1.37 s, total: 1min 29s
Wall time: 1min 29s


In [ ]:
%%time
ids2, val2 = similarity_one_vs_all_pandas(data, Arnold_schwarzenegger_pd)

CPU times: user 10min 29s, sys: 3.14 s, total: 10min 32s
Wall time: 10min 36s
